## DATA 558: Take Home Midterm

Geoffrey Li

May 19, 2019

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import datetime

## Exercise 1

### 1.1: Load, Process Data

Consider the Spam dataset from The Elements of Statistical Learning. Standardize the data, if you have not done so already. Be sure to use the training and test splits from the website.

In [3]:
spam = pd.read_csv('./spam.data', sep='\s+', header=None)
test_indicator = pd.read_csv('./spam.traintest.txt', header=None)

In [4]:
# Create our X matrix with the predictors and y vector with the response
X = np.asarray(spam)[:, 0:-1]
y = np.asarray(spam)[:, -1]*2 - 1 # Convert to +/- 1
test_indicator = np.array(test_indicator).T[0]

In [5]:
# Divide the data into train, test sets
X_train = X[test_indicator == 0, :]
X_test = X[test_indicator == 1, :]
y_train = y[test_indicator == 0][:, np.newaxis]
y_test = y[test_indicator == 1][:, np.newaxis]

y = y[:, np.newaxis]

In [6]:
from sklearn import preprocessing

# Standardize the data
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Keep track of the number of samples and dimension of each sample
n_train = len(y_train)
n_test = len(y_test)
d = np.size(X, 1)

### 1.2: Define Gradient Descent Functions

$\ell_2^{2}$-regularized binary logistic regression with $\rho$-logistic loss

Compute the gradient ∇F(β) of F.

$$\nabla F(\beta) = \frac{1}{n} \sum_{i=1}^{n} -y_i x_i^{T} \frac{exp(-\rho y_ix_i^{T}\beta)}{1+exp(-\rho y_ix_i^{T}\beta)} +2 \lambda \beta $$

In [8]:
def computegrad(x, y, beta, lamb, rho):
    p = np.identity(len(x)) - np.diag(1/(1+np.exp(np.multiply(-rho*y, x@beta))).reshape(1, -1)[0])
    return -1/len(x) * x.T @ p @ y + 2*lamb*beta


def computeobj(x, y, beta, lamb, rho):
    return 1/(len(x)*rho)*np.sum(np.log(1+np.exp(np.multiply(-rho*y, x@beta)))) + lamb*np.sum(beta**2)


def backtracking(curr_beta, lamb, x, y, rho, eta_t=1, alpha=0.5, gamma=0.5, max_iter=100):
    grad_curr_beta = computegrad(x, y, curr_beta, lamb, rho)  # Gradient at current beta
    norm_grad_curr_beta = np.sqrt(np.sum(grad_curr_beta ** 2))  # Norm of the gradient at current beta
    found_eta_t = False
    i = 0  # Iteration counter

    while (found_eta_t is False and i < max_iter):
        if (computeobj(x, y, curr_beta - eta_t * grad_curr_beta, lamb, rho) <
                computeobj(x, y, curr_beta, lamb, rho) - alpha * eta_t * norm_grad_curr_beta ** 2):
            found_eta_t = True
        elif i == max_iter - 1:
            raise ('Maximum number of iterations of backtracking reached')
        else:
            eta_t *= gamma
            i += 1

    return eta_t


def initstepsize(x, lamb):
    return 1/(max(np.linalg.eigh(1/len(x)*x.T@x)[0]) + lamb)


def graddescent(beta_init, lamb, x, y, rho, ss_init, targ_acc):
    beta_values = list()
    beta_values.append(beta_init)

    grad_beta = computegrad(x, y, beta_init, lamb, rho)
    norm_grad_beta = np.sqrt(np.sum(grad_beta ** 2))
    tuned_step_size = ss_init

    t = 0

    while norm_grad_beta > targ_acc:
        tuned_step_size = backtracking(beta_values[t], lamb, x, y, rho, 
                                       eta_t=tuned_step_size, alpha=0.5, gamma=0.8, max_iter=100)
        beta_values.append(beta_values[t] - tuned_step_size * grad_beta)

        t += 1

        grad_beta = computegrad(x, y, beta_values[t], lamb, rho)
        norm_grad_beta = np.sqrt(np.sum(grad_beta ** 2))

    return beta_values


def fastgradalgo(beta_init, lamb, x, y, rho, theta_init, ss_init, targ_acc):
    beta_values = list()
    beta_values.append(beta_init)
    theta = theta_init

    grad_theta = computegrad(x, y, theta, lamb, rho)

    grad_beta = computegrad(x, y, beta_init, lamb, rho)
    norm_grad_beta = np.sqrt(np.sum(grad_beta ** 2))

    tuned_step_size = ss_init

    t = 0

    while norm_grad_beta > targ_acc:
        grad_theta = computegrad(x, y, theta, lamb, rho)

        tuned_step_size = backtracking(beta_values[t], lamb, x, y, rho, 
                                       eta_t=tuned_step_size, alpha=0.5, gamma=0.8, max_iter=100)

        beta_values.append(theta - tuned_step_size * grad_theta)
        theta = beta_values[t + 1] + (t / (t + 3)) * (beta_values[t + 1] - beta_values[t])

        t += 1

        grad_beta = computegrad(x, y, beta_values[t], lamb, rho)
        norm_grad_beta = np.sqrt(np.sum(grad_beta ** 2))

    return beta_values


def misclassificationerror(y, x, b):
    return 1 - np.mean(
        np.fromiter(map(lambda p: 1 if p >= 0.5 else -1, 1/(1+np.exp(-x@b))), 
                    dtype=np.int).reshape(-1,1) == y)




Write a function myrhologistic that implements the accelerated gradient algorithm to train the l2-regularized binary logistic regression with ρ-logistic loss. The function takes as input the initial step-size for the backtracking rule, the ε for the stopping criterion based on the norm of the gradient of the objective, and the value of ρ.

In [9]:
def myrhologistic(x_train, y_train, init_eta, target_accuracy, rho, lamb):
    init_beta = np.zeros(x_train.shape[1])[:, np.newaxis]
    init_theta = np.zeros(x_train.shape[1])[:, np.newaxis]
    
    # Run fast gradient and train classifier
    beta_opt = fastgradalgo(init_beta, lamb, x_train, y_train, rho, init_theta, init_eta, target_accuracy)
    beta_opt_T = beta_opt[len(beta_opt)-1]
    
    return beta_opt_T

### 1.3: Define Cross-Validation Functions

Write a function crossval that implements leave-one-out cross-validation and hold-out cross- validation. You may either write a function that implements each variant separately depend- ing on the case, or write a general cross-validation function that can be instantiated in each case.

In [10]:
def crossval(x, y, testsetsize, params, targ_acc, rho):
    # testsetsize denotes percentage of training data to be split as test data (e.g. 0.20 is 80/20 split)
    # testsetsize = 1 indicates leave-one-out CV
    
    n = len(x)
    cv_results = dict()
    errors = list()
    
    if testsetsize == 1: #leave-one-out CV
        for l in params:
            print('Start CV for Lambda = '+str(l))
            eta_init = initstepsize(x, l)
            errors = list()
            
            for i in range(n):
                cv_x_test = x[i,:]
                cv_y_test = y[i,:]
                cv_x_train = np.delete(x, i, 0)
                cv_y_train = np.delete(y, i, 0)

                beta_T = myrhologistic(cv_x_train, cv_y_train, eta_init, targ_acc, rho, l)
                errors.append(misclassificationerror(cv_y_test, cv_x_test, beta_T))
                print('LOO CV Iteration '+str(i)+' complete.')
            
            cv_results[l] = np.mean(errors)
    elif testsetsize > 0 and testsetsize < 1:
        testset_idx = random.sample(range(n), int(n*testsetsize)) # Generate random test set indices
        cv_x_test = x[testset_idx,:]
        cv_y_test = y[testset_idx,:]
        cv_x_train = np.delete(x, testset_idx, 0)
        cv_y_train = np.delete(y, testset_idx, 0)
        
        for l in params:
            eta_init = initstepsize(x, l)
            
            beta_T = myrhologistic(cv_x_train, cv_y_train, eta_init, targ_acc, rho, l)
            cv_results[l] = misclassificationerror(cv_y_test, cv_x_test, beta_T)
            
    else:
        raise('Invalid test set size.')
    
    return min(cv_results, key=cv_results.get), cv_results

### 1.4 Train Model with $\lambda = 1$

Train your l2-regularized binary logistic regression with ρ-logistic loss with ρ = 2 and ε = 10−3 on the the Spam dataset for the λ = 1. Report your misclassification error for this value of λ.

In [11]:
print(datetime.datetime.now())

# Initialize parameters
lambduh = 1
target_accuracy = 10**-3
r = 2
eta_init = initstepsize(X_train, lambduh)

# Train the Myrhologistic Classifier
beta_T_naive = myrhologistic(X_train, y_train, eta_init, target_accuracy, r, lambduh)

print(datetime.datetime.now())

2019-05-17 02:16:59.017872
2019-05-17 02:17:04.298353


In [12]:
# Compute Misclassification Error
misclassificationerror(y_test, X_test, beta_T_naive)

0.10677083333333337

### 1.5: Train Model: Optimal $\lambda$ using 80/20 Hold-Out CV

Find the optimal value of λ using hold-out cross-validation with a 80%/20% split for the training set/testing set. Report your misclassification errors for the two values of λ found.

In [13]:
print(datetime.datetime.now())

# Initialize parameters
target_accuracy = 10**-3
r = 2
test_set_fraction = 0.20
lambs = [10**i for i in range(-4, 3)]

# Run CV to find optimal lambda
opt_lamb, cv_res_holdout = crossval(X_train, y_train, test_set_fraction, lambs, target_accuracy, r)
eta_init = initstepsize(X_train, opt_lamb)

# Train the Myrhologistic Classifier
beta_T_cv_holdout = myrhologistic(X_train, y_train, eta_init, target_accuracy, r, opt_lamb)



print(datetime.datetime.now())

2019-05-17 02:17:04.318644
2019-05-17 02:20:44.203575


In [14]:
# Compute Misclassification Error
misclassificationerror(y_test, X_test, beta_T_cv_holdout)

0.08268229166666663

Misclassification improved from the $\lambda = 1$ run, as expected.

### 1.6: Train Model: Optimal $\lambda$ using Leave-One-Out CV

Find the optimal value of λ using leave-one-out cross-validation. Report your misclassification errors for the two values of λ found.

In [ ]:
print(datetime.datetime.now())

# Initialize parameters
target_accuracy = 10**-2
r = 2
test_set_fraction = 1
lambs = [10**i for i in range(2, -2, -1)]

# Run CV to find optimal lambda
opt_lamb, cv_res_loo = crossval(X_train, y_train, test_set_fraction, lambs, target_accuracy, r)
eta_init = initstepsize(X_train, opt_lamb)

# Train the Myrhologistic Classifier
beta_T_cv_loo = myrhologistic(X_train, y_train, eta_init, target_accuracy, r, opt_lamb)


print(datetime.datetime.now())

In [21]:
# Compute Misclassification Error
misclassificationerror(y_test, X_test, beta_T_cv_loo)

0.08854166666666663

LOO CV is extremely computationally efficient, as we need to train the model n times for each $\lambda$ value. Despite the extra CV time, the misclassification error isn't even better than 80/20 holdout CV. 